In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.notebook.scroll_to_top();
                jupyter.actions.call('jupyter-notebook:save-notebook');                
                
            });
        });
        
        $( document ).ready(function(){
        code_shown=false;
        $('div.input').hide()});
    
    
</script>

# Overview

This script provides the water satisification requirement index (WSRI) at a given location over time. It uses the downloaded [Famine Land Data Assimilation System](https://ldas.gsfc.nasa.gov/FLDAS/) (FLDAS)* data which can be acquired through the Crop Yield Data Download script. 

This notebook will take you step by step from the data to output.

Contributions to provide improvements are welcome.

\**McNally, A., Arsenault, K., Kumar, S., Shukla, S., Peterson, P., Wang, S., Funk, C., Peters-Lidard, C.D., & Verdin, J. P. (2017). A land data assimilation system for sub-Saharan Africa food and water security applications. Scientific Data, 4, 170012*

## User Tips

- **Every cell ran when the notebook opened**


- **Each input is "live"; entering an input then running the cell will delete your input**


- **After making an input each cell below that input needs to be rerun to use that new input**


- **If you get an error rerun the cells above the cell that errored and it will rectify**

## 0: Python Dependencies 

This cell imports the python dependencies we use to download the data. It has already run, but if click the run cell button, the buttons for this cell will disappear. 

In [2]:
#import libraries
from toggle_code import toggle_code as hide_code
from toggle_code import run_code as run_code

import glob 
from netCDF4 import Dataset
import pandas as pd
import numpy as np2
import math
import pcse
import datetime
import ipywidgets as widgets
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Spectral4
from bokeh.models import Legend, BoxAnnotation, Toggle
from bokeh.layouts import layout, gridplot, column
tile_provider = get_provider('STAMEN_TERRAIN')
#create pyproj transformer to convert form lat/long to web mercator
from pyproj import Transformer
transformer = Transformer.from_crs('epsg:4326','epsg:3857')
#from IPython.display import HTML
output_notebook()
import warnings
warnings.filterwarnings("ignore", message="Cannot find a last shown plot to update.")


Loading BokehJS ...

## 1: Initializing the data

Next, we store the max and min dates from our list of months, so we can create a table (dataframe) with an index for the climate data from each month. We also add the column variables that we want to extract to the dataframe. 

Now we have an empty dataframe to fill with the downloaded data for the variables we need to calculate different crop growths

*If you want to look at all the possible variables to extract from the FLDAS dataset you can open the code and delete the # in front of the print(data.variables.keys()) line below.* 

In [3]:
hide_code()
run_code()
#Retrieve the dates form the downlaoded files to serves as an index for the dataframe
all_months = []
#print("The following dates were downloaded:")
#Retrieves dates to create index list form downloaded files
for file in glob.glob(r'data/*.nc4'):
    #print(file)
    data = Dataset(file, 'r')
    #time = data.variables['time'].units
    #print(time)
    date = file[26:30] + '-' + file[30:32]
    #month = time[11:21]
    all_months.append(date)
    #print(time)
    #print (date)

#Retrieve elevevation data and stores in a dictionary of
# Key = Long, Lat pair
# Value = Elevation
elev_dict = {}
for file in glob.glob(r'data/*.csv'):
    elev = pd.read_csv(file)
    for idx, row in elev.iterrows():
        elev_dict[(row["Latitude"],row["Longitude"])] =row["Elevation"]


# sort months of python list
all_months.sort( )

month_start = min(all_months)
month_end = max(all_months)
#print("Start:", month_start)
#print("Month End:", month_end)
date_range = pd.date_range(start = month_start, end = month_end, freq = 'MS' )

#print(data.variables.keys())

#creates an empty data frame with the dates
df = pd.DataFrame(0.0, columns = ['air_temp' , 'humidity' , 'net_short_radiation', 'net_long_radiation', 'wind_speed', 'evapotranspiration'], index = date_range)
df.index.name = 'Date'

df


,air_temp,humidity,net_short_radiation,net_long_radiation,wind_speed,evapotranspiration
Date,,,,,,
2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-06-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-08-01,0.0,0.0,0.0,0.0,0.0,0.0
2019-09-01,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Get the specific location to assess 

Here is where you can input your desired location to calculate the Water Requirements Satisficiation Index and Crop Yield over time. 

In [4]:
hide_code()
run_code()
file_loc = r"data/FLDAS_NOAH01_C_GL_M.A" + all_months[0][0:4]+all_months[0][5:7] + ".001.nc.SUB.nc4"
data_loc =Dataset(file_loc, 'r')
lat_array = data_loc.variables['Y'][:]#.compressed() # unmasks numpy_array lat stored in the FLDAS files 
long_array = data_loc.variables['X'][:]#.compressed() # unmasks numpy array long stored in FLDAS files
min_lat = lat_array[0]
max_lat = lat_array[-1]
min_long = long_array[0]
max_long = long_array[-1]
long_points = {'globe':[], 'web':[]}
lat_points = {'globe':[], 'web':[]}
for i in range(len(lat_array)):
    for j in range(len(long_array)):
        if round(lat_array[i],2) not in lat_points['globe']:
            lat_points['globe'].append(round(lat_array[i],2))
        if round(long_array[j],2) not in long_points['globe']:
            long_points['globe'].append(round(long_array[j],2))
        pt = transformer.transform(lat_array[i], long_array[j])
        if pt[0] not in long_points['web']: 
            long_points['web'].append(pt[0])
        if pt[1] not in lat_points['web']:
            lat_points['web'].append(pt[1])
            
pts = [(min_lat, min_long), (max_lat, max_long)]
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)       
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
y = [lat_points["web"][50]]
x = [long_points["web"][80]]
loc = p.square(x,y, color= "red", size = 10, alpha = 0.5)


def update(latitude, longitude):
    idx_long = long_points["globe"].index(longitude)
    idx_lat = lat_points["globe"].index(latitude)
    loc.data_source.data['x'] = [long_points["web"][idx_long]]
    loc.data_source.data['y'] = [lat_points["web"][idx_lat]]
    push_notebook()
    return longitude, latitude
    
loc_input = interact(update,latitude=(lat_points["globe"][0], lat_points["globe"][-1]), 
                     longitude=(long_points["globe"][0],long_points["globe"][-1]))

show(p, notebook_handle=True)
print ("  ") #Hides the output

interactive(children=(FloatSlider(value=17.55, description='latitude', max=23.75, min=11.35), FloatSlider(valu…

# 3. Knowing the Data

The next several cells show the data from the selected location and transform it so we can assess crop yields. 

In [5]:
hide_code()
run_code()
# Get user input form sliders
lat_user = loc_input.__dict__["widget"].children[0].__dict__["_trait_values"]["value"]
lon_user = loc_input.__dict__["widget"].children[1].__dict__["_trait_values"]["value"]
#Fill dataframe with data from FLDAS files
df_time_index = 0
for month in all_months:

    file_name = r"data/FLDAS_NOAH01_C_GL_M.A" + month[0:4]+ month[5:7] + ".001.nc.SUB.nc4"
    data = Dataset(file_name, 'r')
    #print(file_name)
    
    # storing the lat and lon data into variables of netCDF file
    lat = data.variables['Y'][:]
    lon = data.variables['X'][:]
    
    #squared diff of lat and lon
    sq_diff_lat = (lat - lat_user)**2
    sq_diff_lon = (lon - lon_user)**2
    
    #getindex of minimum sq difference
    min_index_lat =sq_diff_lat.argmin()
    min_index_lon = sq_diff_lon.argmin()
    
    #accessing the soil moisture data and storing it into the final dataframe
    air_temp = data.variables['Tair_f_tavg']
    humidity = data.variables['Qair_f_tavg']
    net_short_radiation = data.variables['SWdown_f_tavg']
    net_long_radiation = data.variables['Lwnet_tavg']
    wind_speed = data.variables['Wind_f_tavg']
    evapotranspiration = data.variables['Evap_tavg']


    #possibly creating the time range for each month and each iteration
    #start = month
    #d_range = pd.date
   # df.iloc[df_time_index] = SM40[0,min_index_lat, min_index_lon]
    df.air_temp[df_time_index] = air_temp[0,min_index_lat, min_index_lon]
    df.humidity[df_time_index] = humidity[0,min_index_lat, min_index_lon]
    df.net_short_radiation[df_time_index] = net_short_radiation[0,min_index_lat, min_index_lon]
    df.net_long_radiation[df_time_index] = net_long_radiation[0,min_index_lat, min_index_lon]
    df.wind_speed[df_time_index] = wind_speed[0,min_index_lat, min_index_lon]
    df.evapotranspiration[df_time_index] = evapotranspiration[0,min_index_lat, min_index_lon]


    df_time_index += 1



df

,air_temp,humidity,net_short_radiation,net_long_radiation,wind_speed,evapotranspiration
Date,,,,,,
2019-01-01,293.399963,0.001798,232.064621,-134.215424,6.106918,6.994571e-08
2019-02-01,294.313904,0.001667,264.079651,-141.833710,7.030757,5.166653e-08
2019-03-01,300.504974,0.002882,263.064392,-129.702148,7.060035,3.931120e-08
2019-04-01,304.576630,0.003116,297.695587,-148.401169,6.086466,7.066744e-08
2019-05-01,308.262421,0.005147,302.113678,-144.552933,5.055212,3.354468e-07
2019-06-01,307.822449,0.008314,289.569702,-128.817200,4.700864,3.627169e-06
2019-07-01,306.384064,0.010676,283.732910,-113.100624,4.952966,1.154942e-05
2019-08-01,302.612701,0.013974,266.076630,-88.335495,4.617210,2.593919e-05
2019-09-01,304.850006,0.008794,284.523560,-124.317780,5.211111,1.228962e-05


## 4. Converting Units

We are using the [Penman-Monteith model (PM model)](https://en.wikipedia.org/wiki/Penman%E2%80%93Monteith_equation#cite_note-1) to calculate the seasonal crop water requirement. In order to input the variables we need to convert the units from the FLDAS data to units utilized by the Penman-Monteith model. 

In the following table we change: 
- evapotranspiration from kilograms per second to millimeter per day
- wind from 10 meter elevation to 2 meter elevation
- temperature from Kelvin to Celsius

In [6]:
hide_code()
run_code()
#converting units to correct measures for the PM equation
# converts kg/m^2/s to mm/day
def convert_to_mm_day(eot):
    return eot * 86400
df['evapotranspiration'] = df['evapotranspiration'].apply(convert_to_mm_day)

# converts wind speed at 10m elevation to wind speed at a 2m elevation
def convert_wind(speed):
    return speed * 4.87 / math.log ( 67.8 * 10 - 5.42 ) 
df['wind_speed'] = df['wind_speed'].apply(convert_wind)


#converts Kelvin to Celcius
def convert_to_C(temp):
    return temp -273.15
df['air_temp'] = df['air_temp'].apply(convert_to_C)
df

,air_temp,humidity,net_short_radiation,net_long_radiation,wind_speed,evapotranspiration
Date,,,,,,
2019-01-01,20.249963,0.001798,232.064621,-134.215424,4.567676,0.006043
2019-02-01,21.163904,0.001667,264.079651,-141.833710,5.258662,0.004464
2019-03-01,27.354974,0.002882,263.064392,-129.702148,5.280561,0.003396
2019-04-01,31.426630,0.003116,297.695587,-148.401169,4.552379,0.006106
2019-05-01,35.112421,0.005147,302.113678,-144.552933,3.781051,0.028983
2019-06-01,34.672449,0.008314,289.569702,-128.817200,3.516016,0.313387
2019-07-01,33.234064,0.010676,283.732910,-113.100624,3.704576,0.997870
2019-08-01,29.462701,0.013974,266.076630,-88.335495,3.453447,2.241146
2019-09-01,31.700006,0.008794,284.523560,-124.317780,3.897656,1.061824


## 5. Elevation data

We then look up the elevation for our location from the downloaded elevation data for our desired location as an input for the Penman-Monteith model algorithm. 

In [7]:
hide_code()
run_code()
#gets elevation in meters using google elevation api
#might need to square diff it

elevation = elev_dict[(lat_user,lon_user)]
print ("Elevation:", elevation, "meters")


#Adding in dates, months and Vapor pressure
dfvp = pd.DataFrame(0.0, columns = [], index = date_range)
dfvp.index.name = 'Date'
dfvp["dates"] =  pd.date_range(start = month_start, end = month_end, freq = 'MS' ) 
dfvp['month'] = dfvp['dates'].dt.month 

vp = [5.7, 5.7, 6.7, 8.5, 13.2, 15.7, 18.4, 20.4, 17.4, 12.3, 8.1, 6.3]
def months(x , vp):
        return vp[x-1]

dfvp['VP'] = dfvp.apply(lambda row: months(row.month , vp) , axis=1)
df["VP"] = dfvp["VP"]


Elevation: 505.0 meters


## 6. Water Requirement Calculator 

Next, we use the [Python Crop Simulation Environment](https://pcse.readthedocs.io/en/stable/) and specifically the Penman-Monteith algorithm to calculate the Water Requirement Satsification Index (WRSI) for each month in the dataframe adding the WR (water requirement on the far right of the dataframe). 

In [8]:
hide_code()
run_code()
#Calculating WR
df["dates"] = dfvp["dates"]
#pcse.util.penman_monteith(DAY, LAT, ELEV, TMIN, TMAX, AVRAD, VAP, WIND2)
df['WR'] = df.apply(lambda row:  pcse.util.penman_monteith(row.dates, lat_user, elevation, row.air_temp , row.air_temp , row.net_short_radiation, row.VP, row.wind_speed), axis=1)
del df["dates"]
df

,air_temp,humidity,net_short_radiation,net_long_radiation,wind_speed,evapotranspiration,VP,WR
Date,,,,,,,,
2019-01-01,20.249963,0.001798,232.064621,-134.215424,4.567676,0.006043,5.7,5.798230
2019-02-01,21.163904,0.001667,264.079651,-141.833710,5.258662,0.004464,5.7,6.587709
2019-03-01,27.354974,0.002882,263.064392,-129.702148,5.280561,0.003396,6.7,8.369192
2019-04-01,31.426630,0.003116,297.695587,-148.401169,4.552379,0.006106,8.5,8.386663
2019-05-01,35.112421,0.005147,302.113678,-144.552933,3.781051,0.028983,13.2,7.435077
2019-06-01,34.672449,0.008314,289.569702,-128.817200,3.516016,0.313387,15.7,6.517200
2019-07-01,33.234064,0.010676,283.732910,-113.100624,3.704576,0.997870,18.4,5.877076
2019-08-01,29.462701,0.013974,266.076630,-88.335495,3.453447,2.241146,20.4,4.137020
2019-09-01,31.700006,0.008794,284.523560,-124.317780,3.897656,1.061824,17.4,5.817277


## 7. Calculating final WRSI (Water Requirement Satisfaction Index)

We use evapotranspiration and WR to calculate the final Water Requirement Satisfaction Index for each month

In [9]:
hide_code()
run_code()
#Calculate final WRSI
df['WRSI'] = df.apply(lambda row:  100* (row.evapotranspiration/row.WR), axis=1)
df
     

,air_temp,humidity,net_short_radiation,net_long_radiation,wind_speed,evapotranspiration,VP,WR,WRSI
Date,,,,,,,,,
2019-01-01,20.249963,0.001798,232.064621,-134.215424,4.567676,0.006043,5.7,5.798230,0.104227
2019-02-01,21.163904,0.001667,264.079651,-141.833710,5.258662,0.004464,5.7,6.587709,0.067762
2019-03-01,27.354974,0.002882,263.064392,-129.702148,5.280561,0.003396,6.7,8.369192,0.040583
2019-04-01,31.426630,0.003116,297.695587,-148.401169,4.552379,0.006106,8.5,8.386663,0.072802
2019-05-01,35.112421,0.005147,302.113678,-144.552933,3.781051,0.028983,13.2,7.435077,0.389809
2019-06-01,34.672449,0.008314,289.569702,-128.817200,3.516016,0.313387,15.7,6.517200,4.808620
2019-07-01,33.234064,0.010676,283.732910,-113.100624,3.704576,0.997870,18.4,5.877076,16.979024
2019-08-01,29.462701,0.013974,266.076630,-88.335495,3.453447,2.241146,20.4,4.137020,54.172942
2019-09-01,31.700006,0.008794,284.523560,-124.317780,3.897656,1.061824,17.4,5.817277,18.252930


## 8. Crop specific Water Requirements

Finally, we calculate a general WRSI. However, we can find the crop specific WRSI data based on how much water a crop requires at diffent stages of its growth. We used the crop co-efficients from the online [United Nations Food and Agricultural Guidelines for Computing Crop Water Requirements](http://www.fao.org/3/X0490e/x0490e0b.htm#crop%20coefficients). 

### Please pick your crops and their life cycle(s) of interest.

* Plant - Water requirements at planting
* Grow - Water requirements during growth
* Harvest - Water requirements at harvest time

(Shift for group of selections; Ctrl-Shift for multiple individual selections)

In [10]:
hide_code()
run_code()
crop_human = widgets.SelectMultiple(
    options = ["cowpeas : plant", "cowpeas : grow", "cowpeas : harvest", 
               "spring wheat : plant", "spring wheat : grow", "spring wheat : harvest",
              "ground nut : plant", "ground nut : grow", "ground nut : harvest", 
              "maize : plant", "maize : grow", "maize  : harvest", 
              "millet : plant", "millet : grow", "millet : harvest",
              "sorghum : plant", "sorghum : grow", "sorghum : harvest"],
    description='Crops',
    disabled=False
)

crop_selected = []
def update(crops):
    crop_selected = list(crops)
    return crop_selected

crops_selection = interact(update, crops= crop_human)


grazing = widgets.SelectMultiple(
    options = ["rotated grazing : grow", "rotated grazing : harvest",
               "extensive grazing : grow", "extensive grazing : harvest"], 
    description="Grazing", 
    disabled = False)


graze = []
def g_update(grazing):
    graze_selected = list(grazing)
    return graze_selected
    
graze_selection = interact(g_update, grazing = grazing)

interactive(children=(SelectMultiple(description='Crops', options=('cowpeas : plant', 'cowpeas : grow', 'cowpe…

interactive(children=(SelectMultiple(description='Grazing', options=('rotated grazing : grow', 'rotated grazin…

In [11]:
hide_code()
run_code()

crops = list(crops_selection.__dict__["widget"].children[0].__dict__["_trait_values"]["value"])
grazes = list(graze_selection.__dict__["widget"].children[0].__dict__["_trait_values"]["value"])

crop_dict = {"cowpeas":[0.4, 1.05,0.6],
            "maize": [0.7,1.20, .35], 
             "millet":[.7,1.20, .35], 
             "sorghum":[.7, 1.20, .35], 
             "spring wheat": [0.4, 1.15,0.33], 
             "ground nut": [0.4, 1.15, 0.6], 
             "rotated grazing": [None, 1.05, 0.85],
             "extensive grazing": [None,0.75,0.75]}

#cowpeas,spring wheat, ground nut and grazing have variable requirements...figure out how to deal with that. 

reqs = {}

def parse_crops(food):
    '''
    param: list of user inputs from crop and gazing selection 
    output: dictionary with crop and lifecycle times
    '''
   
    
    for f in food: 
        inputs = f.split(':')
        crop = inputs[0][0:-1] # get rid of final space
        cycle = inputs[1][1:] #get rid of lead space
                    
        if crop not in reqs.keys(): 
            reqs[crop] =[cycle]
        else: 
            reqs[crop].append(cycle)
    
    return reqs

reqs = parse_crops(crops)
reqs = parse_crops(grazes)


#Initializing empty data frame
dfcrops = pd.DataFrame(0.0, columns = [], index = date_range)
dfcrops.index.name = 'Date'

def get_cycle(cycle):
    cycle_idx = ""
    if cycle == "plant":
        cycle_idx = 0
    elif cycle == "grow":
        cycle_idx = 1
    else: 
        cycle_idx = 2
    return cycle_idx

def make_crop_df(reqs):
    
    for crop,cycle in reqs.items(): 
        for c in cycle: 
            idx = get_cycle(c)
            dfcrops[crop+" " + c] = df["WRSI"]/crop_dict[crop][idx]

make_crop_df(reqs)
dfcrops

""
Date
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01


## Plotting Data

Now we can plot the final WRSI(Seasonal Water Requirement Satisfaction Index) models for each crop/crop stage. The WRSI model calculates the crop yield in relation to a water deficit. It follows the scale

100+: Very Good

95-100: Good

80-94: Average

60-79: Mediocre

50-59: Poor

In [12]:
hide_code()
run_code()

columns = list(dfcrops.columns)

plots = widgets.SelectMultiple(
    options = columns, 
    description="Data to Plot", 
    disabled = False)

title = widgets.Text(
    value='Crop Yields',
    description='Title',
    disabled=False
)

font = widgets.IntText(
    value=10,
    description='Font Size',
    disabled=False
)

size = widgets.IntText(
    value=600,
    description='Plot Size',
    disabled=False
)

to_plot = []
def plot_update(plots, title, font, size):
    to_plot = list(plots)
    font_label = str(font)+"pt"
    axis_label = str(font*1.5) + "pt"
    title_label = str(font*2)+"pt"
    leg = []
    ph = size
    if len(to_plot) == 0:
        pass
    else: 
        above = 0
        p = figure(plot_width=size, plot_height=ph, title=title, x_range = list(dfcrops.index.astype(str)))
        p.xaxis.major_label_orientation = 45
        p.title.text_font_size = title_label
        p.xaxis.axis_label_text_font_size = font_label
        p.xaxis.axis_label = "Date"
        p.yaxis.axis_label = "Water Satisfication"
        for plot, color in zip(to_plot, Spectral4): 
            l = p.line(list(dfcrops.index.astype(str)), dfcrops[plot], color = color, line_width = 2)
            if max(dfcrops[plot]) > above:
                above = max(dfcrops[plot])
            leg.append((plot, [l]))
        legend = Legend(items= leg, location=(0,ph/2))
        green_box = BoxAnnotation(top=above+10, bottom=94, fill_color='green', fill_alpha=0.2)
        yellow_box = BoxAnnotation(top=94, bottom=80, fill_color='yellow', fill_alpha=0.2)
        orange_box = BoxAnnotation(top=80, bottom=50, fill_color='orange', fill_alpha=0.2)
        red_box = BoxAnnotation(top=50, bottom=0, fill_color='red', fill_alpha=0.2)
        boxes = [green_box, yellow_box, orange_box, red_box]
        p.add_layout(legend, "right")
       
        for box in boxes: 
            p.add_layout(box)
                
        toggle1 = Toggle(label="Good Water", button_type="success", active=True)
        toggle1.js_link('active', green_box, 'visible')
        
        toggle2 = Toggle(label= "Average Water", button_type="success", active=True)
        toggle2.js_link('active', yellow_box, 'visible')
        
        toggle3 = Toggle(label="Mediocre Water", button_type="success", active=True)
        toggle3.js_link('active', orange_box, 'visible')
        
        toggle4 = Toggle(label="Poor Water", button_type="success", active=True)
        toggle4.js_link('active', red_box, 'visible')
        #inputs = column(plots, title, font, size)
        show(layout([p], [toggle1, toggle2], [toggle3, toggle4]))
            

plot_selection = interact(plot_update, plots = plots, title = title, font = font, size = size)

interactive(children=(SelectMultiple(description='Data to Plot', options=(), value=()), Text(value='Crop Yield…